# 🔫 Weapon Detection & Analysis Pipeline

**Vision-Language Model (VLM) powered weapon detection and analysis**

This notebook provides a complete pipeline for:
1. **Object Detection** - Using YOLOv11 to detect objects in images
2. **Cropping** - Extracting detected regions with expanded bounding boxes
3. **VLM Analysis** - Using Qwen2.5-VL to analyze each crop for weapon identification

---
**Designed for Google Colab** | GPU recommended for faster inference


## 📦 Step 1: Install Dependencies

Run this cell to install all required packages. This only needs to be run once per session.


In [ ]:
# Install required packages (Google Colab)
!pip install -q ultralytics opencv-python matplotlib pillow
!pip install -q transformers accelerate qwen-vl-utils
!pip install -q torch torchvision torchaudio

print("✅ All dependencies installed!")


## 📚 Step 2: Import Libraries

Import all necessary libraries for the pipeline.


In [ ]:
# ── Standard library ────────────────────────────────────────────
import os
import math
import random
import textwrap
import io
import requests

# ── Numerical computing ─────────────────────────────────────────
import numpy as np

# ── Deep-learning stack ─────────────────────────────────────────
import torch
from transformers import (
    Qwen2_5_VLForConditionalGeneration,
    AutoProcessor,
)

# ── Imaging & visualization ─────────────────────────────────────
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# ── Object detection ────────────────────────────────────────────
from ultralytics import YOLO

# ── Project-specific helpers ────────────────────────────────────
from qwen_vl_utils import process_vision_info

# ── Colab-specific ──────────────────────────────────────────────
try:
    from google.colab import files
    from google.colab.patches import cv2_imshow
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

print(f"🔧 Running in Google Colab: {IN_COLAB}")
print(f"🖥️ PyTorch version: {torch.__version__}")
print(f"🎮 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")


## ⚙️ Step 3: Configuration

Set up configuration parameters for the pipeline. Modify these as needed.


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# CONFIGURATION - Modify these parameters as needed
# ══════════════════════════════════════════════════════════════════════════════

class Config:
    """Configuration class for the weapon detection pipeline."""
    
    # ── YOLO Detection Settings ──
    YOLO_MODEL = "yolo11l.pt"           # Options: yolo11n.pt, yolo11s.pt, yolo11m.pt, yolo11l.pt
    DETECTION_CONFIDENCE = 0.1          # Lower = more detections (0.0 to 1.0)
    DETECTION_IOU = 0.5                 # IoU threshold for NMS
    
    # ── Cropping Settings ──
    EXPAND_RATIO = 0.1                  # Expand bounding boxes by this ratio (0.1 = 10%)
    
    # ── VLM Settings ──
    VLM_MODEL_ID = "Qwen/Qwen2.5-VL-3B-Instruct"  # Vision-Language Model
    VLM_MAX_TOKENS = 64                 # Max tokens for VLM generation
    
    # ── Visualization Settings ──
    GRID_COLS = 4                       # Number of columns in result grid
    FIG_SIZE_MULTIPLIER = 5             # Figure size multiplier
    
    # ── Default Prompt ──
    DEFAULT_PROMPT = (
        "Describe the weapon detected in the photo. Be very specific, "
        "type of weapon (eg handgun), brand if possible and if unsure don't say a brand. "
        "Describe the intent of the user with the gun, are they actively going to use it or are they passive. "
        "If unsure say that you are unsure unless certain."
    )

config = Config()
print("✅ Configuration loaded!")
print(f"   YOLO Model: {config.YOLO_MODEL}")
print(f"   VLM Model: {config.VLM_MODEL_ID}")
print(f"   Detection Confidence: {config.DETECTION_CONFIDENCE}")


## 🛠️ Step 4: Utility Functions

Define reusable utility functions for image loading, visualization, and analysis.


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# UTILITY FUNCTIONS
# ══════════════════════════════════════════════════════════════════════════════

def load_image(image_source):
    """
    Load an image from various sources (file path, URL, or PIL Image).
    
    Args:
        image_source: str (file path or URL) or PIL.Image
    
    Returns:
        tuple: (PIL Image, OpenCV BGR image, height, width)
    """
    if isinstance(image_source, Image.Image):
        image_pil = image_source
    elif image_source.startswith(('http://', 'https://')):
        # Load from URL
        response = requests.get(image_source)
        image_pil = Image.open(io.BytesIO(response.content))
        print(f"📥 Downloaded image from URL")
    else:
        # Load from file path
        image_pil = Image.open(image_source)
        print(f"📂 Loaded image from: {image_source}")
    
    # Convert to RGB if needed
    if image_pil.mode != 'RGB':
        image_pil = image_pil.convert('RGB')
    
    # Convert PIL to OpenCV BGR format
    image_cv = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)
    h_img, w_img = image_cv.shape[:2]
    
    print(f"   Image size: {w_img} x {h_img} pixels")
    
    return image_pil, image_cv, h_img, w_img


def display_image(image, title="Image", figsize=(10, 10)):
    """Display a PIL image with matplotlib."""
    plt.figure(figsize=figsize)
    plt.imshow(image)
    plt.axis("off")
    plt.title(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()


def display_image_grid(images, titles=None, cols=4, figsize_multiplier=4):
    """
    Display multiple images in a grid.
    
    Args:
        images: List of PIL Images
        titles: Optional list of titles for each image
        cols: Number of columns in the grid
        figsize_multiplier: Size multiplier for the figure
    """
    num = len(images)
    rows = math.ceil(num / cols)
    
    plt.figure(figsize=(cols * figsize_multiplier, rows * figsize_multiplier))
    
    for i, img in enumerate(images):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(img)
        plt.axis("off")
        if titles and i < len(titles):
            plt.title(titles[i], fontsize=10)
        else:
            plt.title(f"Image {i}", fontsize=10)
    
    plt.tight_layout()
    plt.show()


def display_detections(image_pil, results, title="YOLO Detections"):
    """
    Visualize YOLO detection results on the original image.
    
    Args:
        image_pil: Original PIL Image
        results: YOLO results object
        title: Title for the plot
    """
    fig, ax = plt.subplots(1, figsize=(12, 12))
    ax.imshow(image_pil)
    
    colors = plt.cm.tab10.colors
    
    for r in results:
        for idx, box in enumerate(r.boxes):
            x1, y1, x2, y2 = map(float, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            
            color = colors[idx % len(colors)]
            
            rect = patches.Rectangle(
                (x1, y1), x2 - x1, y2 - y1,
                linewidth=2,
                edgecolor=color,
                facecolor='none'
            )
            ax.add_patch(rect)
            
            # Add label
            label = f"#{idx} conf:{conf:.2f}"
            ax.text(x1, y1 - 5, label, fontsize=9, color='white',
                   bbox=dict(boxstyle='round,pad=0.3', facecolor=color, alpha=0.8))
    
    ax.axis("off")
    ax.set_title(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print(f"📊 Detected {len(r.boxes)} objects")

print("✅ Utility functions defined!")


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# CROPPING FUNCTIONS
# ══════════════════════════════════════════════════════════════════════════════

def extract_crops(image_cv, results, expand_ratio=0.1):
    """
    Extract cropped regions from YOLO detection results.
    
    Args:
        image_cv: OpenCV BGR image
        results: YOLO results object
        expand_ratio: How much to expand bounding boxes (0.1 = 10%)
    
    Returns:
        list: List of PIL Images (cropped regions)
    """
    h_img, w_img = image_cv.shape[:2]
    pil_crops = []
    crop_info = []  # Store metadata about each crop
    
    for r in results:
        for idx, box in enumerate(r.boxes):
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            
            w = x2 - x1
            h = y2 - y1
            
            # Expand bounding box
            dx = int(w * expand_ratio / 2)
            dy = int(h * expand_ratio / 2)
            
            x1_new = max(0, x1 - dx)
            y1_new = max(0, y1 - dy)
            x2_new = min(w_img, x2 + dx)
            y2_new = min(h_img, y2 + dy)
            
            # Crop using OpenCV (BGR)
            crop_bgr = image_cv[y1_new:y2_new, x1_new:x2_new]
            
            # Convert BGR → RGB → PIL
            crop_rgb = cv2.cvtColor(crop_bgr, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(crop_rgb)
            
            pil_crops.append(pil_img)
            crop_info.append({
                'index': idx,
                'confidence': conf,
                'bbox_original': (x1, y1, x2, y2),
                'bbox_expanded': (x1_new, y1_new, x2_new, y2_new),
                'size': (x2_new - x1_new, y2_new - y1_new)
            })
    
    print(f"✂️ Extracted {len(pil_crops)} crops from detections")
    return pil_crops, crop_info

print("✅ Cropping functions defined!")


## 🧠 Step 5: VLM Analysis Functions

Define the Vision-Language Model analysis functions for weapon detection.


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# VLM ANALYSIS FUNCTIONS
# ══════════════════════════════════════════════════════════════════════════════

def analyze_crop_with_vlm(crop_img, processor, vlm_model, device, prompt_text=None, max_tokens=64):
    """
    Analyze a single crop image with the Vision-Language Model.
    
    Args:
        crop_img: PIL Image to analyze
        processor: AutoProcessor instance
        vlm_model: Qwen2_5_VLForConditionalGeneration model
        device: Device string ("cuda" or "cpu")
        prompt_text: Optional custom prompt. If None, uses default weapon analysis prompt.
        max_tokens: Maximum tokens for generation
    
    Returns:
        str: Analysis text/caption
    """
    if prompt_text is None:
        prompt_text = config.DEFAULT_PROMPT
    
    msgs = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": crop_img},
                {"type": "text", "text": prompt_text}
            ],
        }
    ]
    
    text_prompt = processor.apply_chat_template(
        msgs,
        tokenize=False,
        add_generation_prompt=True
    )
    
    image_inputs, video_inputs = process_vision_info(msgs)
    
    # Pack text + vision into model-ready tensors
    inputs = processor(
        text=[text_prompt],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    ).to(device)
    
    # Run inference (no gradients, pure generation)
    with torch.no_grad():
        generated_ids = vlm_model.generate(
            **inputs,
            max_new_tokens=max_tokens
        )
    
    # Extract the newly generated tokens (skip the prompt length)
    caption = processor.batch_decode(
        generated_ids[:, inputs.input_ids.shape[-1]:],
        skip_special_tokens=True
    )[0]
    
    return caption


def analyze_all_crops(pil_crops, processor, vlm_model, device, prompt_text=None, show_progress=True):
    """
    Analyze all crops with the VLM.
    
    Args:
        pil_crops: List of PIL Images
        processor: AutoProcessor instance
        vlm_model: Vision-Language Model
        device: Device string
        prompt_text: Optional custom prompt
        show_progress: Whether to show progress updates
    
    Returns:
        list: List of (crop_image, analysis_text) tuples
    """
    results = []
    total = len(pil_crops)
    
    if show_progress:
        print(f"\n🔍 Analyzing {total} crops with VLM...")
        print("─" * 50)
    
    for i, crop in enumerate(pil_crops):
        if show_progress:
            print(f"  [{i+1}/{total}] Processing crop {i}...", end=" ")
        
        analysis = analyze_crop_with_vlm(
            crop, processor, vlm_model, device, 
            prompt_text=prompt_text,
            max_tokens=config.VLM_MAX_TOKENS
        )
        results.append((crop, analysis))
        
        if show_progress:
            # Show preview of analysis
            preview = analysis[:60] + "..." if len(analysis) > 60 else analysis
            print(f"✓")
            print(f"      → {preview}")
    
    if show_progress:
        print("─" * 50)
        print(f"✅ Completed analysis of {total} crops!\n")
    
    return results

print("✅ VLM analysis functions defined!")


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# RESULTS DISPLAY FUNCTIONS
# ══════════════════════════════════════════════════════════════════════════════

def display_analysis_results(analysis_results, cols=4, figsize_multiplier=5):
    """
    Display analysis results in a grid with images and text.
    
    Args:
        analysis_results: List of (crop_image, analysis_text) tuples
        cols: Number of columns in the grid
        figsize_multiplier: Size multiplier for the figure
    """
    num_results = len(analysis_results)
    rows = math.ceil(num_results / cols)
    
    # Create figure with enough space for images and text
    fig, axes = plt.subplots(rows, cols, figsize=(cols * figsize_multiplier, rows * (figsize_multiplier + 1)))
    
    # Flatten axes for easy iteration
    if rows == 1 and cols == 1:
        axes = [[axes]]
    elif rows == 1:
        axes = [axes]
    elif cols == 1:
        axes = [[ax] for ax in axes]
    
    for i, (img, analysis_text) in enumerate(analysis_results):
        row = i // cols
        col = i % cols
        ax = axes[row][col] if rows > 1 else axes[0][col] if cols > 1 else axes[0][0]
        
        ax.imshow(img)
        ax.axis("off")
        
        # Wrap the analysis text for display
        wrapped_text = textwrap.fill(analysis_text, width=35)
        ax.set_title(f"Crop {i}\n{wrapped_text}", fontsize=8, pad=5, wrap=True)
    
    # Hide empty subplots
    for i in range(num_results, rows * cols):
        row = i // cols
        col = i % cols
        ax = axes[row][col] if rows > 1 else axes[0][col] if cols > 1 else axes[0][0]
        ax.axis("off")
        ax.set_visible(False)
    
    plt.tight_layout()
    plt.show()


def print_full_analysis(analysis_results):
    """
    Print complete analysis results in a formatted way.
    
    Args:
        analysis_results: List of (crop_image, analysis_text) tuples
    """
    print("\n" + "═" * 80)
    print("📋 COMPLETE ANALYSIS RESULTS")
    print("═" * 80)
    
    for i, (img, analysis_text) in enumerate(analysis_results):
        print(f"\n┌─ Crop {i} {'─' * 68}")
        wrapped_caption = textwrap.fill(analysis_text, width=76, initial_indent="│ ", subsequent_indent="│ ")
        print(wrapped_caption)
        print(f"└{'─' * 77}")
    
    print("\n" + "═" * 80)
    print(f"Total crops analyzed: {len(analysis_results)}")
    print("═" * 80)

print("✅ Results display functions defined!")


## 🔄 Step 6: Complete Pipeline Class

A unified class that encapsulates the entire detection and analysis pipeline.


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# WEAPON DETECTION PIPELINE CLASS
# ══════════════════════════════════════════════════════════════════════════════

class WeaponDetectionPipeline:
    """
    Complete pipeline for weapon detection and analysis.
    
    Usage:
        pipeline = WeaponDetectionPipeline()
        pipeline.load_models()
        results = pipeline.analyze_image("path/to/image.jpg")
        pipeline.display_results(results)
    """
    
    def __init__(self, config=None):
        """Initialize the pipeline with optional custom config."""
        self.config = config or Config()
        self.yolo_model = None
        self.vlm_model = None
        self.processor = None
        self.device = None
        self.models_loaded = False
    
    def load_models(self):
        """Load both YOLO and VLM models."""
        print("🚀 Loading models...")
        print("─" * 50)
        
        # Load YOLO model
        print(f"   Loading YOLO: {self.config.YOLO_MODEL}...")
        self.yolo_model = YOLO(self.config.YOLO_MODEL)
        print("   ✓ YOLO loaded")
        
        # Set device
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"   Device: {self.device}")
        
        # Load VLM model
        print(f"   Loading VLM: {self.config.VLM_MODEL_ID}...")
        self.vlm_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
            self.config.VLM_MODEL_ID,
            torch_dtype="auto",
            device_map="auto"
        )
        self.processor = AutoProcessor.from_pretrained(self.config.VLM_MODEL_ID)
        print("   ✓ VLM loaded")
        
        self.models_loaded = True
        print("─" * 50)
        print("✅ All models loaded successfully!\n")
    
    def detect_objects(self, image_path, visualize=True):
        """
        Run YOLO detection on an image.
        
        Args:
            image_path: Path to image or URL
            visualize: Whether to display detection visualization
        
        Returns:
            tuple: (image_pil, image_cv, results, h_img, w_img)
        """
        if not self.models_loaded:
            raise RuntimeError("Models not loaded. Call load_models() first.")
        
        # Load image
        image_pil, image_cv, h_img, w_img = load_image(image_path)
        
        if visualize:
            display_image(image_pil, title="Original Image", figsize=(10, 10))
        
        # Run YOLO detection
        print("\n🔎 Running YOLO detection...")
        results = self.yolo_model(
            image_path if isinstance(image_path, str) else np.array(image_pil),
            conf=self.config.DETECTION_CONFIDENCE,
            iou=self.config.DETECTION_IOU,
            verbose=False,
        )
        
        if visualize:
            display_detections(image_pil, results, title="YOLO Detection Results")
        
        return image_pil, image_cv, results, h_img, w_img
    
    def analyze_image(self, image_path, prompt=None, visualize_steps=True):
        """
        Complete analysis pipeline: detect → crop → analyze.
        
        Args:
            image_path: Path to image or URL
            prompt: Optional custom prompt for VLM
            visualize_steps: Whether to show intermediate visualizations
        
        Returns:
            dict: Complete analysis results
        """
        if not self.models_loaded:
            raise RuntimeError("Models not loaded. Call load_models() first.")
        
        print("═" * 60)
        print("🔫 WEAPON DETECTION & ANALYSIS PIPELINE")
        print("═" * 60)
        
        # Step 1: Detect objects
        image_pil, image_cv, yolo_results, h_img, w_img = self.detect_objects(
            image_path, visualize=visualize_steps
        )
        
        # Step 2: Extract crops
        print("\n✂️ Extracting detected regions...")
        pil_crops, crop_info = extract_crops(
            image_cv, yolo_results, 
            expand_ratio=self.config.EXPAND_RATIO
        )
        
        if visualize_steps and pil_crops:
            titles = [f"Crop {i} ({info['confidence']:.2f})" for i, info in enumerate(crop_info)]
            display_image_grid(pil_crops, titles=titles, cols=self.config.GRID_COLS)
        
        # Step 3: Analyze with VLM
        if pil_crops:
            analysis_results = analyze_all_crops(
                pil_crops, self.processor, self.vlm_model, self.device,
                prompt_text=prompt, show_progress=True
            )
        else:
            print("⚠️ No objects detected to analyze.")
            analysis_results = []
        
        # Compile results
        results = {
            'image_pil': image_pil,
            'yolo_results': yolo_results,
            'crops': pil_crops,
            'crop_info': crop_info,
            'analysis_results': analysis_results,
            'num_detections': len(pil_crops)
        }
        
        return results
    
    def display_results(self, results, show_grid=True, print_text=True):
        """
        Display analysis results.
        
        Args:
            results: Results dict from analyze_image()
            show_grid: Whether to show image grid
            print_text: Whether to print full text analysis
        """
        if not results['analysis_results']:
            print("⚠️ No results to display.")
            return
        
        if show_grid:
            display_analysis_results(
                results['analysis_results'],
                cols=self.config.GRID_COLS,
                figsize_multiplier=self.config.FIG_SIZE_MULTIPLIER
            )
        
        if print_text:
            print_full_analysis(results['analysis_results'])

print("✅ WeaponDetectionPipeline class defined!")


## 🎯 Step 7: Load Image

Load your image for analysis. You can use a file path, URL, or upload from your computer.


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# OPTION A: Download sample image from URL
# ══════════════════════════════════════════════════════════════════════════════

# Sample image URL (you can replace with your own)
IMAGE_URL = "https://static0.moviewebimages.com/wordpress/wp-content/uploads/article/YFQdcXrsu64BJMMhEl6k2WPjorzYG4.jpg"

# Download the image
!wget -q -O sample_image.jpg "{IMAGE_URL}"
IMAGE_PATH = "sample_image.jpg"

print(f"📥 Downloaded sample image to: {IMAGE_PATH}")


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# OPTION B: Upload your own image (Colab only)
# ══════════════════════════════════════════════════════════════════════════════

# Uncomment the following lines to upload your own image in Google Colab:

# from google.colab import files
# uploaded = files.upload()
# IMAGE_PATH = list(uploaded.keys())[0]
# print(f"📤 Uploaded: {IMAGE_PATH}")


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# PREVIEW THE IMAGE
# ══════════════════════════════════════════════════════════════════════════════

# Load and display the image
image_pil, image_cv, h_img, w_img = load_image(IMAGE_PATH)
display_image(image_pil, title=f"Input Image ({w_img}x{h_img})", figsize=(12, 12))


## 🚀 Step 8: Initialize and Load Models

Initialize the pipeline and load the YOLO and VLM models. This may take a few minutes on first run.


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# INITIALIZE PIPELINE AND LOAD MODELS
# ══════════════════════════════════════════════════════════════════════════════

# Create pipeline instance
pipeline = WeaponDetectionPipeline(config)

# Load all models (YOLO + VLM)
pipeline.load_models()


## 🔍 Step 9: Run Detection & Analysis

Run the complete pipeline on your image. This will:
1. Detect objects using YOLO
2. Crop detected regions
3. Analyze each crop with the VLM


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# RUN COMPLETE ANALYSIS
# ══════════════════════════════════════════════════════════════════════════════

# Run the full pipeline with visualization at each step
results = pipeline.analyze_image(IMAGE_PATH, visualize_steps=True)

print(f"\n📊 Summary: Detected and analyzed {results['num_detections']} objects")


## 📊 Step 10: Display Final Results

View the complete analysis results with visualizations and detailed text output.


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# DISPLAY FINAL RESULTS
# ══════════════════════════════════════════════════════════════════════════════

# Display results grid with analysis text
pipeline.display_results(results, show_grid=True, print_text=True)


## 🔧 Advanced: Analyze Individual Crops

You can also analyze individual crops with custom prompts.


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# ANALYZE SINGLE CROP WITH CUSTOM PROMPT
# ══════════════════════════════════════════════════════════════════════════════

# Select a specific crop to analyze (change index as needed)
if results['crops']:
    crop_index = 0  # Change this to analyze different crops
    selected_crop = results['crops'][crop_index]
    
    # Display the selected crop
    display_image(selected_crop, title=f"Selected Crop {crop_index}", figsize=(6, 6))
    
    # Custom prompt example
    custom_prompt = "Describe this image in detail. What objects do you see? What is the context?"
    
    # Analyze with custom prompt
    custom_analysis = analyze_crop_with_vlm(
        selected_crop,
        pipeline.processor,
        pipeline.vlm_model,
        pipeline.device,
        prompt_text=custom_prompt,
        max_tokens=128  # More tokens for detailed response
    )
    
    print("📝 Custom Analysis Result:")
    print("─" * 50)
    print(textwrap.fill(custom_analysis, width=70))
else:
    print("⚠️ No crops available to analyze.")


## 📁 Alternative: Step-by-Step Manual Execution

If you prefer more control, you can run each step manually instead of using the pipeline class.


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# MANUAL STEP-BY-STEP EXECUTION (Alternative to Pipeline)
# ══════════════════════════════════════════════════════════════════════════════
# Uncomment the code below to run steps manually

"""
# ── Step 1: Load YOLO Model ──
yolo_model = YOLO("yolo11l.pt")

# ── Step 2: Load Image ──
image_path = "your_image.jpg"
image_pil, image_cv, h_img, w_img = load_image(image_path)
display_image(image_pil, title="Original Image")

# ── Step 3: Run YOLO Detection ──
yolo_results = yolo_model(image_path, conf=0.1, iou=0.5, verbose=False)
display_detections(image_pil, yolo_results)

# ── Step 4: Extract Crops ──
pil_crops, crop_info = extract_crops(image_cv, yolo_results, expand_ratio=0.1)
display_image_grid(pil_crops, cols=4)

# ── Step 5: Load VLM Model ──
device = "cuda" if torch.cuda.is_available() else "cpu"
vlm_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

# ── Step 6: Analyze Crops ──
analysis_results = analyze_all_crops(pil_crops, processor, vlm_model, device)

# ── Step 7: Display Results ──
display_analysis_results(analysis_results)
print_full_analysis(analysis_results)
"""

print("💡 Uncomment the code above to run the pipeline step-by-step manually.")


## 🧹 Cleanup (Optional)

Free up GPU memory when done.


In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# CLEANUP - Free GPU memory
# ══════════════════════════════════════════════════════════════════════════════

# Uncomment to free GPU memory when done:

# del pipeline
# del results
# torch.cuda.empty_cache()
# print("🧹 GPU memory cleared!")
